# Format DataFrame values

See the [styling docs](https://pandas.pydata.org/pandas-docs/stable/style.html) especially, [this section](https://pandas.pydata.org/pandas-docs/stable/style.html#Finer-Control:-Display-Values).

In [1]:
# Setup

import pandas as pd

## Create toy data

In [2]:
# Create toy data

data = [
    (1000, .624324324),
    (10000, .91241)
]

df = pd.DataFrame.from_records(data, columns=['big_number', 'pct'])

## Use `.style.format`

By default, Pandas doesn't format numeric values when displaying a DataFrame.

In [3]:
# Show default styling of numbers

df

,big_number,pct
0,1000,0.624324
1,10000,0.912410


However you can specify a dictionary mapping column names to [Python format strings](https://pyformat.info).

In [4]:
# Use style.format

df.style.format({
    'big_number': '{:,}',
    'pct': '{:.1%}',
})

,big_number,pct
0,"1,000",62.4%
1,"10,000",91.2%
